In [4]:
from mip import Model, xsum, maximize, BINARY, CONTINUOUS, OptimizationStatus
import random
import pandas as pd

In [5]:
resources = ['Alice','Bob','Carl','Dan','Eme','Fran','Gustav']
skills = ['Engineer','Operation','Design']
proyects = range(10)
days = range(31)

In [143]:
matrix_skill = [[0 for j in range(len(resources))] for i in range(len(skills))]

for j in range(len(resources)):
    fila_aleatoria = random.randint(0, len(skills)-1)
    matrix_skill[fila_aleatoria][j] = 1
    
matrix_skill = pd.DataFrame(index=skills, columns= resources, data=matrix_skill)
matrix_skill

,Alice,Bob,Carl,Dan,Eme,Fran,Gustav
Engineer,1,1,0,1,0,0,0
Operation,0,0,0,0,0,0,1
Design,0,0,1,0,1,1,0


In [144]:
project_time = {}
project_benefit = {}
for p in proyects:
    project_benefit[p] = random.randint(1,10)
    skill_time = {}
    for s in skills:
        skill_time[s]=random.randint(10,30)
    project_time[p]=skill_time

In [145]:
project_benefit

{0: 3, 1: 4, 2: 10, 3: 6, 4: 9, 5: 2, 6: 10, 7: 6, 8: 9, 9: 6}

In [146]:
capacity_person_hh = {}
for i in resources:
    capacity_person_hh[i] = 60

In [147]:
class RCPSP():
    def __init__(self) -> None:
        self.model = Model('Task Schedeling')

    def sets(self, resources, skills, proyects):
        self.resources = resources
        self.skills = skills
        self.proyects = proyects

    def parameters(self, matrix_skill, project_time, project_benefit, capacity_person_hh):
        self.matrix_skill = matrix_skill
        self.project_time = project_time
        self.project_benefit = project_benefit
        self.capacity_person_hh = capacity_person_hh

    def variables(self):
        self.x = [[[self.model.add_var(var_type=CONTINUOUS, name=f"x_{r}_{s}_{p}") for p in self.proyects] for s in self.skills] for r in self.resources]
        self.y = [[self.model.add_var(var_type=BINARY, name=f"Project_{p}_skill_{s}_complete") for s in self.skills ]for p in self.proyects]
        self.z = [self.model.add_var(var_type=BINARY, name=f"Project_{p}_Complete") for p in self.proyects]

    def constrains(self):
        # un colaborador no trabaja más de su horas maximas
        for r, resource in enumerate(resources):
            self.model += xsum(self.x[r][s][p] for s in range(len(skills)) for p in self.proyects ) <= self.capacity_person_hh[resource]

        # colaborador solo puede realizar sus skills
        for p in self.proyects:
            for r, resource in enumerate(resources):
                for s, skill in enumerate(self.skills):
                    self.model += self.x[r][s][p] <= self.matrix_skill.loc[skill][resource] * 1000

        # relación x e y
        for p in self.proyects:
            for s, skill in enumerate(self.skills):
              self.model += xsum(self.x[r][s][p] for r in range(len(self.resources))) >= self.y[p][s] * self.project_time[p][skill]

        # Relación y , z
        for p in self.proyects:
            self.model += self.y[p][0] >= self.z[p]
            self.model +=  self.y[p][1] >= self.z[p]
            self.model +=  self.y[p][2] >= self.z[p]

    def objetive(self):
        self.model.objective = maximize((xsum(self.z[p] * self.project_benefit[p] for p in self.proyects) ))

    def run_model(self):
        status = self.model.optimize()

        print("#############################################")
        if status == OptimizationStatus.OPTIMAL:
            print('[OPTIMAL] optimal solution profit {} found'.format(self.model.objective_value))
        elif status == OptimizationStatus.FEASIBLE:
            print('[FEASIBLE]  sol.profit {} found, best possible: {}'.format(self.model.objective_value, self.model.objective_bound))
        elif status == OptimizationStatus.NO_SOLUTION_FOUND:
            print('[Not Found] no feasible solution found, lower bound is: {}'.format(self.model.objective_bound))

In [148]:
modelo = RCPSP()
modelo.sets(resources, skills, proyects)
modelo.parameters(matrix_skill, project_time, project_benefit, capacity_person_hh)
modelo.variables()
modelo.constrains()
modelo.objetive()

In [84]:
modelo.run_model()

#############################################
[OPTIMAL] optimal solution profit 23.0 found


In [137]:
result = []
for z, Z in enumerate(modelo.z):
    # print(z.x)
    if Z.x==1:
        print("_________________")
        for r, res in enumerate(resources):
            for s, skill in enumerate(skills):
                if matrix_skill.loc[skill][res]==1:
                    if modelo.x[r][s][z].x > 0:
                        print(modelo.x[r][s][z].name)
                        print(modelo.x[r][s][z].x)
                        result.append([str(z),res,skill,modelo.x[r][s][z].x])
            

_________________
x_Alice_Engineer_3
60.0
x_Carl_Operation_3
17.0
x_Eme_Design_3
17.0
_________________
x_Bob_Engineer_5
36.0
x_Carl_Operation_5
26.0
x_Eme_Design_5
12.0
x_Gustav_Design_5
14.0
_________________
x_Carl_Operation_8
17.0
x_Eme_Design_8
15.0
x_Fran_Engineer_8
14.0


In [127]:
project_time[8]

{'Engineer': 10, 'Operation': 17, 'Design': 15}

In [86]:
modelo.x[1][0][6].name

'x_Bob_Engineer_6'

In [138]:
column = ['proyect', 'person','skill','time']
df = pd.DataFrame(data=result,columns=column)
df

,proyect,person,skill,time
0,3,Alice,Engineer,60.0
1,3,Carl,Operation,17.0
2,3,Eme,Design,17.0
3,5,Bob,Engineer,36.0
4,5,Carl,Operation,26.0
5,5,Eme,Design,12.0
6,5,Gustav,Design,14.0
7,8,Carl,Operation,17.0
8,8,Eme,Design,15.0
9,8,Fran,Engineer,14.0


In [142]:
import plotly.express as px

fig = px.bar(df, x="proyect", y="time",
             color='person', barmode='stack',
             height=700)
fig.show()

In [129]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


,proyect,person,skill,time
0,3,Alice,Engineer,60.0
1,3,Carl,Operation,17.0
2,3,Eme,Design,17.0
3,5,Bob,Engineer,36.0
4,5,Carl,Operation,26.0
5,5,Eme,Design,12.0
6,5,Gustav,Design,14.0
7,8,Carl,Operation,17.0
8,8,Eme,Design,15.0
9,8,Fran,Engineer,14.0
